In [1]:
import pandas as pd
import numpy as np
import datetime
import time

In [2]:
from tools.pos_encoder import *
from tools.time_encoder import *

In [3]:
from IPython.display import display, HTML
def show_df(df):
    display(HTML(df.to_html()))

# Read

In [4]:
train_dataset_path = '../data/eleme_round1_train/'
test_dataset_path = '../data/eleme_round1_testA/'
testB_dataset_path = '../data/eleme_round1_testB/'

In [5]:
def read_a_day(dataset_path, read_date):
    df_action = pd.read_csv('%saction/action_%s.txt' % (dataset_path, read_date.strftime('%Y%m%d')))
    df_courier = pd.read_csv('%scourier/courier_%s.txt' % (dataset_path, read_date.strftime('%Y%m%d')))
    df_distance = pd.read_csv('%sdistance/distance_%s.txt' % (dataset_path, read_date.strftime('%Y%m%d')))
    df_order = pd.read_csv('%sorder/order_%s.txt' % (dataset_path, read_date.strftime('%Y%m%d')))
    return df_action, df_courier, df_distance, df_order

def read_days(start_date, end_date, data_dir):
    cur_date = start_date
    actions = []
    couriers = []
    distances = []
    orders = []
    while cur_date < end_date:
        df_action, df_courier, df_distance, df_order = read_a_day(data_dir, cur_date)
        df_action['date'] = cur_date
        df_courier['date'] = cur_date
        df_distance['date'] = cur_date
        df_order['date'] = cur_date
        
        actions.append(df_action)
        couriers.append(df_courier)
        distances.append(df_distance)
        orders.append(df_order)
        cur_date += datetime.timedelta(days = 1)
    
    df_actions = pd.concat(actions, axis = 0, ignore_index = True)
    df_couriers = pd.concat(couriers, axis = 0, ignore_index = True)    
    df_distances = pd.concat(distances, axis = 0, ignore_index = True)    
    df_orders = pd.concat(orders, axis = 0, ignore_index = True) 
    return df_actions, df_couriers, df_distances, df_orders

In [6]:
df_actions_train, df_couriers_train, df_distances_train, df_orders_train = read_days(datetime.datetime(2020, 2, 1), datetime.datetime(2020, 3, 1), train_dataset_path)
df_actions_testA, df_couriers_testA, df_distances_testA, df_orders_testA = read_days(datetime.datetime(2020, 3, 1), datetime.datetime(2020, 3, 7), test_dataset_path)

In [13]:
df_actions_testB, df_couriers_testB, df_distances_testB, df_orders_testB = read_days(datetime.datetime(2020, 3, 1), datetime.datetime(2020, 3, 7), testB_dataset_path)

In [7]:
df_orders_testA

,courier_id,wave_index,tracking_id,weather_grade,pick_lng,pick_lat,deliver_lng,deliver_lat,create_time,confirm_time,assigned_time,promise_deliver_time,estimate_pick_time,aoi_id,shop_id,date
0,100006950,0,2100076028337521770,正常天气,121.435172,39.045104,121.445424,39.030201,1583031080,1583031081,1583031099,1583034560,1583032341,a22e938ecbcbda978b3130ce921f3539,71ab63c19a2b6ef5924e809a540d1d00,2020-03-01
1,100006950,0,2100076029547709715,正常天气,121.437147,39.045412,121.426724,39.031871,1583031280,1583031283,1583031339,1583034760,1583032303,2a81d869ad23907ae4f094cbf9a7144a,0bbcad93810f203c750220e055cf28a3,2020-03-01
2,100006950,0,2100076031141348641,正常天气,121.436453,39.043794,121.440344,39.044991,1583031472,1583031473,1583031519,1583033932,1583032253,79563b239ee269b542b03336ecce97a6,309e9cefd957115647ce992354946f6e,2020-03-01
3,100006950,0,2100076032019399862,正常天气,121.437105,39.045363,121.431104,39.038801,1583031733,1583031733,1583031759,1583035213,1583032933,619567be776f3cf9764522c693bc1946,d3f8b520e8a23b6c2c5c1120414744cb,2020-03-01
4,100006950,0,2100076030888641642,正常天气,121.435172,39.045104,121.444334,39.037091,1583031845,1583031846,1583031940,1583035025,1583032746,717afa9d93a87f20ccee19fe193122b5,71ab63c19a2b6ef5924e809a540d1d00,2020-03-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18799,125779698,1,2100076513371719254,轻微恶劣天气,121.608369,39.181012,121.600364,39.185541,1583488497,1583488497,1583488501,1583491497,1583489577,cd7019da955d597fd63de054fc52b6ee,8ebd589f62d84bee56a310021bb41d13,2020-03-06
18800,125779698,1,2100076513672267350,轻微恶劣天气,121.608369,39.181012,121.602344,39.193581,1583488566,1583488567,1583488621,1583491686,1583489827,ccde0a11e918d116ba43b2be58b0da8c,8ebd589f62d84bee56a310021bb41d13,2020-03-06
18801,125812730,1,2100076455950288622,正常天气,121.409921,39.125993,121.403944,39.125741,1583463185,1583463186,1583463245,1583464986,1583464026,bc5c9a185daab157203aba08ea886d52,ada39f980c647b532fc65b60935ef574,2020-03-06
18802,125812730,1,2100076456506066671,正常天气,121.416328,39.130422,121.412454,39.126301,1583463340,1583463341,1583463365,1583465141,1583464121,6d301a8f38f0ff4437e812b6039c0f3c,33f24e06a738cff391f0c2435589f9ff,2020-03-06


# build dataset-level features

## one hot

In [7]:
def one_hot_features(df_train, df_test, one_hot_features):
    df_train['type'] = 'train'
    df_test['type'] = 'test'
    one_hot_names = []
    df = pd.concat([df_train, df_test])
    for feature in one_hot_features:
        df_one_hot = pd.get_dummies(df[feature], prefix=feature)
        df = pd.concat([df, df_one_hot], axis=1)
        one_hot_names.extend(list(df_one_hot.columns))
    return df.query('type == "train"'), df.query('type == "test"'), one_hot_names

In [8]:
df_actions_train, df_actions_testA, actions_one_hot_names = one_hot_features(df_actions_train, df_actions_testA, ['action_type'])
df_couriers_train, df_couriers_testA, couriers_one_hot_names = one_hot_features(df_couriers_train, df_couriers_testA, ['level', 'max_load'])
df_distances_train, df_distances_testA, distances_one_hot_names = one_hot_features(df_distances_train, df_distances_testA, ['source_type', 'target_type'])
df_orders_train, df_orders_testA, orders_one_hot_names = one_hot_features(df_orders_train, df_orders_testA, ['weather_grade'])

In [9]:
df_couriers_train['level'].value_counts()

3    13550
2     5754
1     1361
0      626
Name: level, dtype: int64

## order aoi shop courier 交叉

In [10]:
df_order = pd.concat([df_orders_train, df_orders_testA])

def build_aoi_feature(df):
    res_dict = {'aoi_count' : df.shape[0]}
    res_dict['aoi_shop_count'] = len(set(df['shop_id']))
    res_dict['aoi_courier_count'] = len(set(df['courier_id']))
    return pd.Series(res_dict)

def build_shop_feature(df):
    res_dict = {'shop_count' : df.shape[0]}
    res_dict['shop_aoi_count'] = len(set(df['aoi_id']))
    res_dict['shop_courier_count'] = len(set(df['courier_id']))
    return pd.Series(res_dict)

df_feature_aoi = df_order.groupby('aoi_id').apply(build_aoi_feature)
df_feature_shop = df_order.groupby('shop_id').apply(build_shop_feature)
df_feature_courier = df_order.groupby('courier_id').apply(lambda x : pd.Series({
    'courier_aoi_count' : len(set(x['aoi_id'])),
    'courier_shop_count' : len(set(x['shop_id'])),
    'courier_count' : x.shape[0]
}))

def add_aoi_shop_features(df_origin, df_feature, origin_name):
    feature_names = df_feature.columns
    for feature in feature_names:
        df_origin[feature] = df_origin[origin_name].map(df_feature[feature])
    return df_origin


df_order = add_aoi_shop_features(df_order, df_feature_aoi, 'aoi_id')
df_order = add_aoi_shop_features(df_order, df_feature_shop, 'shop_id')
df_order = add_aoi_shop_features(df_order, df_feature_courier, 'courier_id')

df_orders_train, df_orders_testA = df_order.query('type == "train"'), df_order.query('type == "test"')

# Map

In [11]:
def get_map(df):
    """
    将数据映射成字典，可通过
        mp[date][courier_id][wave_index]
    访问

    """
    last_date, last_courier, last_wave = df.at[0, 'date'], df.at[0, 'courier_id'], df.at[0, 'wave_index']
    last_i = 0
    mp_date, mp_courier, mp_wave = {}, {}, {}
    key_list = []
    for i in range(1, df.shape[0]):
        cur_wave = df.at[i, 'wave_index']
        cur_courier = df.at[i, 'courier_id']
        cur_date = df.at[i, 'date']

        if last_wave != cur_wave or (last_wave == cur_wave and last_courier != cur_courier):
            mp_wave[last_wave] = df.iloc[last_i : i]
            last_i = i
            key_list.append([last_date, last_courier, last_wave])
            last_wave = cur_wave

        if last_courier != cur_courier or (last_courier == cur_courier and last_date != cur_date):
            mp_courier[last_courier] = mp_wave.copy()
            mp_wave = {}
            last_courier = cur_courier

        if last_date != cur_date:
            mp_date[last_date] = mp_courier.copy()
            mp_courier = {}
            last_date = cur_date
    
    mp_wave[last_wave] = df.iloc[last_i : ]
    key_list.append([last_date, last_courier, last_wave])
    mp_courier[last_courier] = mp_wave.copy()
    mp_date[last_date] = mp_courier.copy()
    mp_courier = {}
    
    return mp_date, key_list


In [12]:
start = time.time()
mp_action_train, action_key_list_train = get_map(df_actions_train)
mp_distance_train, distance_key_list_train = get_map(df_distances_train)
mp_order_train, order_key_list_train = get_map(df_orders_train)
time.time() - start

629.2347650527954

In [13]:
start = time.time()
mp_action_testA, action_key_list_testA = get_map(df_actions_testA)
mp_distance_testA, distance_key_list_testA = get_map(df_distances_testA)
mp_order_testA, order_key_list_testA = get_map(df_orders_testA)
time.time() - start

19.60161542892456

In [14]:
start = time.time()
mp_action_testB, action_key_list_testB = get_map(df_actions_testB)
mp_distance_testB, distance_key_list_testB = get_map(df_distances_testB)
mp_order_testB, order_key_list_testB = get_map(df_orders_testB)
time.time() - start

25.775644302368164

In [16]:
def get_map_distance_detail(mp_distance, key_list):
    mp_distance_detail = {}
    for (date, courier, wave_idx) in key_list:
        if date not in mp_distance_detail:
            mp_distance_detail[date] = {}
        if courier not in mp_distance_detail[date]:
            mp_distance_detail[date][courier] = {}
        if wave_idx not in mp_distance_detail[date][courier]:
            mp_distance_detail[date][courier][wave_idx] = {}
        
        df_a_distance = mp_distance[date][courier][wave_idx]
        
        gby_tracking = df_a_distance.groupby('tracking_id')
        for tracking_id, df_track_id in gby_tracking:
            mp_distance_detail[date][courier][wave_idx][tracking_id] = {}
            gby_target_tracking = df_track_id.groupby('target_tracking_id')
            for target_trackingid, df_target_tracking in gby_target_tracking:
                mp_distance_detail[date][courier][wave_idx][tracking_id][target_trackingid] = df_target_tracking
            
    return mp_distance_detail

In [15]:
start = time.time()
mp_distance_detail_train = get_map_distance_detail(mp_distance_train, action_key_list_train)
mp_distance_detail_testA = get_map_distance_detail(mp_distance_testA, action_key_list_testA)

time.time() - start

839.732988357544

In [17]:
mp_distance_detail_testB = get_map_distance_detail(mp_distance_testB, action_key_list_testB)


# Check

In [16]:
len(action_key_list_train), len(distance_key_list_train), len(order_key_list_train), \
len(action_key_list_testA), len(distance_key_list_testA), len(order_key_list_testA)

(82533, 82533, 82533, 4219, 4219, 4219)

In [18]:
len(action_key_list_testB), len(distance_key_list_testB), len(order_key_list_testB)

(4193, 4193, 4193)

In [17]:
sorted(action_key_list_train) == sorted(distance_key_list_train) == sorted(order_key_list_train), \
sorted(action_key_list_testA) == sorted(distance_key_list_testA) == sorted(order_key_list_testA)

(True, True)

In [19]:
sorted(action_key_list_testB) == sorted(distance_key_list_testB) == sorted(order_key_list_testB)

True

In [18]:
date = datetime.datetime(2020, 3, 6)
df_actions_testA.query('date == @date & courier_id == "10325553"') 

,courier_id,wave_index,tracking_id,courier_wave_start_lng,courier_wave_start_lat,action_type,expect_time,date,type,action_type_DELIVERY,action_type_PICKUP
37602,10325553,1,2100076463172683062,121.460578,39.154954,PICKUP,1583466111,2020-03-06,test,0,1
37603,10325553,1,2100076464220046355,121.460578,39.154954,PICKUP,1583466269,2020-03-06,test,0,1
37604,10325553,1,2100076464812787347,121.460578,39.154954,DELIVERY,0,2020-03-06,test,1,0
37605,10325553,1,2100076463172683062,121.460578,39.154954,DELIVERY,0,2020-03-06,test,1,0
37606,10325553,1,2100076464220046355,121.460578,39.154954,DELIVERY,0,2020-03-06,test,1,0
37607,10325553,1,2100076464812787347,121.460578,39.154954,PICKUP,0,2020-03-06,test,0,1


In [19]:
date = datetime.datetime(2020, 3, 6)
mp_action_testA[date][10325553]

{1:        courier_id  wave_index          tracking_id  courier_wave_start_lng  \
 37602    10325553           1  2100076463172683062              121.460578   
 37603    10325553           1  2100076464220046355              121.460578   
 37604    10325553           1  2100076464812787347              121.460578   
 37605    10325553           1  2100076463172683062              121.460578   
 37606    10325553           1  2100076464220046355              121.460578   
 37607    10325553           1  2100076464812787347              121.460578   
 
        courier_wave_start_lat action_type  expect_time       date  type  \
 37602               39.154954      PICKUP   1583466111 2020-03-06  test   
 37603               39.154954      PICKUP   1583466269 2020-03-06  test   
 37604               39.154954    DELIVERY            0 2020-03-06  test   
 37605               39.154954    DELIVERY            0 2020-03-06  test   
 37606               39.154954    DELIVERY            0 2020-0

In [20]:
i = 0
for (a, b, c), (aa, bb, cc) in zip(sorted(action_key_list_testA), sorted(distance_key_list_testA)):
    if a != aa or b != bb or c != cc:
        print(i)
    i += 1
        

# pickle

In [20]:
import pickle
PICKLE_PATH = '../user_data/data_sort_out'

In [22]:
testB_dataset_mp = {
    'key_list' : action_key_list_testB,
    'mp_action' : mp_action_testB,
    'mp_distance' : mp_distance_testB,
    'mp_order' : mp_order_testB,
    'mp_distance_detail' : mp_distance_detail_testB
}
with open('%s/testB_dataset_mp.pickle'%(PICKLE_PATH), 'wb') as f:
    pickle.dump(testB_dataset_mp, f)

In [23]:
testB_dataset_df = {
    'df_actions' : df_actions_testB,
    'df_couriers' : df_couriers_testB,
    'df_distances' : df_distances_testB,
    'df_orders' : df_orders_testB
}
with open('%s/testB_dataset_df.pickle'%(PICKLE_PATH), 'wb') as f:
    pickle.dump(testB_dataset_df, f)

In [22]:
train_dataset_mp = {
    'key_list' : action_key_list_train,
    'mp_action' : mp_action_train,
    'mp_distance' : mp_distance_train,
    'mp_order' : mp_order_train,
    'mp_distance_detail' : mp_distance_detail_train
}
with open('%s/train_dataset_mp.pickle'%(PICKLE_PATH), 'wb') as f:
    pickle.dump(train_dataset_mp, f)

In [23]:
testA_dataset_mp = {
    'key_list' : action_key_list_testA,
    'mp_action' : mp_action_testA,
    'mp_distance' : mp_distance_testA,
    'mp_order' : mp_order_testA,
    'mp_distance_detail' : mp_distance_detail_testA
}
with open('%s/testA_dataset_mp.pickle'%(PICKLE_PATH), 'wb') as f:
    pickle.dump(testA_dataset_mp, f)

In [24]:
train_dataset_df = {
    'df_actions' : df_actions_train,
    'df_couriers' : df_couriers_train,
    'df_distances' : df_distances_train,
    'df_orders' : df_orders_train
}
with open('%s/train_dataset_df.pickle'%(PICKLE_PATH), 'wb') as f:
    pickle.dump(train_dataset_df, f)

In [25]:
testA_dataset_df = {
    'df_actions' : df_actions_testA,
    'df_couriers' : df_couriers_testA,
    'df_distances' : df_distances_testA,
    'df_orders' : df_orders_testA
}
with open('%s/testA_dataset_df.pickle'%(PICKLE_PATH), 'wb') as f:
    pickle.dump(testA_dataset_df, f)

In [26]:
one_hot_name = {
    'actions' : actions_one_hot_names,
    'couriers' : couriers_one_hot_names,
    'distances' : distances_one_hot_names,
    'orders' : orders_one_hot_names
}
with open('%s/one_hot_name.pickle'%(PICKLE_PATH), 'wb') as f:
    pickle.dump(one_hot_name, f)